In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

try:
  from google.colab  import  drive
  drive.mount('/content/gdrive')
  mydirectory = 'gdrive/My Drive/Colab Notebooks/PD/'
except:
  mydirectory = ''

#Parkinson's dataset is downloaded from UCI ML repository: https://archive.ics.uci.edu/ml/datasets/parkinsons
fname=mydirectory+"parkinsons.data"       #195 rows/recordings, there are only 32 human subjects (8 of which are PD)
data = pd.read_csv(fname)                 #6 recordings per subject (7 recordings for only 3 of the subjects)
dataX = data.drop(columns=['status','name'])   #We have 6records*29subj + 7records*3subjects = 195
#print the first few rows
print(data)


               name  MDVP:Fo(Hz)  MDVP:Fhi(Hz)  MDVP:Flo(Hz)  MDVP:Jitter(%)  \
0    phon_R01_S01_1      119.992       157.302        74.997         0.00784   
1    phon_R01_S01_2      122.400       148.650       113.819         0.00968   
2    phon_R01_S01_3      116.682       131.111       111.555         0.01050   
3    phon_R01_S01_4      116.676       137.871       111.366         0.00997   
4    phon_R01_S01_5      116.014       141.781       110.655         0.01284   
..              ...          ...           ...           ...             ...   
190  phon_R01_S50_2      174.188       230.978        94.261         0.00459   
191  phon_R01_S50_3      209.516       253.017        89.488         0.00564   
192  phon_R01_S50_4      174.688       240.005        74.287         0.01360   
193  phon_R01_S50_5      198.764       396.961        74.904         0.00740   
194  phon_R01_S50_6      214.289       260.277        77.973         0.00567   

     MDVP:Jitter(Abs)  MDVP:RAP  MDVP:P

In [0]:
#FIRST ATTEMPT
#NOTE THAT THIS IS NOT A GOOD FIT WITH THE DATASET, WE NEED LEAVE SUBJECT OUT INSTEAD

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(dataX, data['status'], test_size=0.5, random_state=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)   #ranges, mean, stddev of the features in the training set
X_test = scaler.transform(X_test) 
#especially if you are going to use rbf-svm, this preprocessing step is very important
#because SVM will be calculateing the distance of the test example to each one of the training examples
#you dont want to have one feature ranging 100 to 200 and the other one (which is perhaps more important) ranging from 0 to 1
#Refer to the discussion in the next code cell about what normalization is.



#Now, we start training-optimizing-testing the classifier
#Find the optimal parameters by cross-validation, well, optimal only if you do a proper cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-1, 1, 10, 100],   #libsvm showed that each point has some influence around itself in the space
                     #gamma is the inverse of the sigma of the "sphere"  (the mean of that sphere is on that particular training example)
                     'C': [0.25, 0.5, 1, 2, 4, 8, 16, 32]},
                    {'kernel': ['linear'], 'C': [0.25, 0.5, 1, 2, 4, 8, 16, 32]}]

clf_plain_split = GridSearchCV(SVC(), tuned_parameters, scoring='f1_macro')  #accuracy can be bad for this dataset because it is imbalanced, we have a lot more Parkinson's subjects.
#Here is the summary of the lecture on why the dataset is imbalanced, thus forcing us to use average of f1 scores of the classes. 
#Keep in mind the study is for telediagnosis of Parkinson's (a good story for COVID-19 era). 
#A good control group is needed, unfortunately we have only 8 healthy subjects.
#In the video I explained why that might be in more detail, the main idea is that finding a good control group (vs Parkinson's) is not easy. 
#You cannot use students as a control group because normally they are not going to be the calling the telediagnosis service to be tested for Parkinsonism. 
#It is important to match demographic groups etc.


clf_plain_split.fit(X_train, y_train)
print("Best parameters set found by plain_split:",clf_plain_split.best_params_)
y_pred = clf_plain_split.predict(X_test)
print(classification_report(y_test, y_pred))

#WOW, we get 91% accuracy
#THAT 91% IS NOT REALISTIC THOUGH!  I EXPLAINED WHY IN THE VIDEO.

Best parameters set found by plain_split: {'C': 2, 'gamma': 0.1, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.94      0.67      0.78        24
           1       0.90      0.99      0.94        74

    accuracy                           0.91        98
   macro avg       0.92      0.83      0.86        98
weighted avg       0.91      0.91      0.90        98



In [0]:
#A LITTLE DISCUSSION ABOUT WHAT NORMALIZATION IS:
#zero centered and unit standard deviation

X_normalized = scaler.fit_transform(dataX)   #ranges, mean, stddev etc of the features in the training set
print('original vs normalized values for the first feature, first 10 rows are shown:')
print(np.concatenate((X_normalized[:10,[0]],dataX.iloc[:10,[0]]), axis=1))
#print(X_normalized[:10,0])
#print(dataX.iloc[:10,0])


original vs normalized values for the first feature, first 10 rows are shown:
[[ -0.82929965 119.992     ]
 [ -0.77097169 122.4       ]
 [ -0.90947638 116.682     ]
 [ -0.90962172 116.676     ]
 [ -0.92565706 116.014     ]
 [ -0.81573501 120.552     ]
 [ -0.82263845 120.267     ]
 [ -1.13595747 107.332     ]
 [ -1.4169878   95.73      ]
 [ -1.43331382  95.056     ]]


In [0]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

logo = LeaveOneGroupOut()
print('N_subjects', logo.get_n_splits(dataX, data['status'],data['name'].astype(str).str[:-2]))

#SPLIT INDIVIDUALS
trains = []
tests = []
N_train = 0
#logo = leave one group out (in this case a group is the set of all sound recordings that belong to one subject)
for train_index, test_index in logo.split(dataX, data['status'], data['name'].astype(str).str[:-2]):
    #print((train_index, test_index))
    if np.random.random()>0.25 :      #if you use 0.5 here 16 randomly selected subjects will be placed in the training set, the rest will be placed in the test set
        trains=np.append(trains, test_index)    #test_index is the index of all recordings for the next subject of the logo iterator
        N_train=N_train+1
    else :
        tests=np.append(tests, test_index)

X_train, X_test = dataX.iloc[trains,:], dataX.iloc[tests,:]
y_train, y_test = data['status'][trains], data['status'][tests]

# Set the parameters by cross-validation
tuned_parameters = [{'svm__kernel': ['rbf'], 'svm__gamma': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100],
                     'svm__C': [0.25, 0.5, 1, 2, 4, 8, 16, 32]},
                    {'svm__kernel': ['linear'], 'svm__C': [0.25, 0.5, 1, 2, 4, 8, 16, 32]}]

from sklearn.pipeline import Pipeline
pipeline = Pipeline([('scale',StandardScaler()), ('svm',SVC())])

#CROSS VALIDATION IN GRIDSEARCH BELOW IS NOT USING LEAVE SUBJECT OUT, IT USES THE STANDARD K-FOLD
#THAT BETTER BE FIXED (FOR THE SAME REASON WHY 91% WAS NOT REALISTIC ABOVE)
#X_train contains 16 subjects but GridSearch will split it into cv folds (cv = 5, 10 or whatever number that you choose)
#Ideally, you want cv = 16 and you want your gridsearch split this set so that one group (=subject) is left out!
clf_logo_split = GridSearchCV(pipeline, tuned_parameters, scoring='f1_macro', cv = N_train)
#[([7,8,9,10,11,12.....],[1,2,3,4,5,6]),([1,2,3,4,5,6,13,14,15,...],[7,8,9,10,11,12]),(),(),...]
#Even if cv = N_train = 16 that's not enough: Gridsearch will leave out 6 examples but they will belong to possibly different subjects
clf_logo_split.fit(X_train, y_train)
print("Best parameters set found by logo_split:",clf_logo_split.best_params_)
y_pred = clf_logo_split.predict(X_test)
print(classification_report(y_test, y_pred))


N_subjects 32
Best parameters set found by logo_split: {'svm__C': 4, 'svm__gamma': 0.1, 'svm__kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.38      0.42      0.40        12
           1       0.80      0.78      0.79        36

    accuracy                           0.69        48
   macro avg       0.59      0.60      0.59        48
weighted avg       0.70      0.69      0.69        48



In [0]:
#SIMPLY SETTING CV=16 WOULD APPLY CROSS VALIDATION WITHOUT LEAVE SUBJECT OUT
#THE BEST SOLUTION WOULD BE TO LEAVE-SUBJECT-OUT (LOGO: LEAVE-GROUP-OUT, see the code below)
#NOTE THAT THE BEST SOLUTION IS NOT THE ONE REPORTING THE HIGHEST ACCURACY
#THE BEST SOLUTION IS THE MOST REALISTIC ONE, THE ONE THAT DOES NOT OVERESTIMATE THE ACCURACY
mycv = logo.split(X_train, y_train, data['name'][trains].astype(str).str[:-2])
clf_logo_split_logo_cv = GridSearchCV(pipeline, tuned_parameters, scoring='f1_macro', cv = mycv)
clf_logo_split_logo_cv.fit(X_train, y_train)
print("Best parameters set found by logo_split_logo_CV:",clf_logo_split_logo_cv.best_params_)
y_pred = clf_logo_split_logo_cv.predict(X_test)
print(classification_report(y_test, y_pred))


Best parameters set found by logo_split_logo_CV: {'svm__C': 16, 'svm__gamma': 0.001, 'svm__kernel': 'rbf'}
              precision    recall  f1-score   support

           0       1.00      0.50      0.67        12
           1       0.86      1.00      0.92        36

    accuracy                           0.88        48
   macro avg       0.93      0.75      0.79        48
weighted avg       0.89      0.88      0.86        48



In [0]:
pipeline.named_steps['svm'].set_params(**clf_plain_split.best_params_)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

pipeline.set_params(**clf_logo_split.best_params_)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

pipeline.set_params(**clf_logo_split_logo_cv.best_params_)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.36      0.33      0.35        12
           1       0.78      0.81      0.79        36

    accuracy                           0.69        48
   macro avg       0.57      0.57      0.57        48
weighted avg       0.68      0.69      0.68        48

              precision    recall  f1-score   support

           0       0.38      0.42      0.40        12
           1       0.80      0.78      0.79        36

    accuracy                           0.69        48
   macro avg       0.59      0.60      0.59        48
weighted avg       0.70      0.69      0.69        48

              precision    recall  f1-score   support

           0       1.00      0.50      0.67        12
           1       0.86      1.00      0.92        36

    accuracy                           0.88        48
   macro avg       0.93      0.75      0.79        48
weighted avg       0.89      0.88      0.86        48

